#Предварительная обработка данных в питоне

Если файлы уже обработаны, то нужно удалить!

In [1]:
import pandas as pd

In [2]:
def prepare_data(file_name, name, columns):
  temp_df = pd.read_csv(file_name)
  temp_df.drop(columns = columns, inplace = True)
  temp_df.rename(columns = {"<CLOSE>": name}, inplace=True)
  return temp_df

In [3]:
columns = ["<TIME>", "<PER>", "<TICKER>"]
file_names = ["BZ_220324_240311.csv", "CA_220324_240311.csv", "EURUSD_220324_240311.csv", "GC_220324_240311.csv", "SPX_220324_240311.csv", "USDRUB_220324_240311.csv"]
names = ["BZ", "CA", "EURUSD", "GC", "SPX", "USDRUB"]

In [4]:
df = prepare_data("GAZP_220324_240311.csv", "GAZP", columns)

for file_name, name in zip(file_names, names):
  df = pd.merge(
      df,
      prepare_data(file_name, name, columns),
      on="<DATE>"
  )
df.drop_duplicates(subset=["<DATE>"], inplace=True)
df.dropna(inplace=True)
# df

In [5]:
df.to_csv("Stocks.csv", index = False)

В новом файле остались только цены закрытия всех стоимостей акций

#Далее идет само задание

In [1]:
install.packages("xts")
library(xts)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: zoo


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric




In [2]:
Data<-read.csv("./Stocks.csv",header = TRUE)
names(Data)[1] <- "DATE"
Data$DATE <- as.Date(Data$DATE, format = "%d/%m/%Y")
Data <- Data[order(Data$DATE ),]
head(Data)

,DATE,GAZP,BZ,CA,EURUSD,GC,SPX,USDRUB
,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,22-03-24,258.51,118.20,10415.77,1.10083,1961.5,4520.16,102.2056
2,22-03-25,227.00,119.98,10339.71,1.09820,1957.6,4543.06,101.2500
3,22-03-29,208.00,108.56,10491.83,1.10910,1923.0,4631.60,85.0000
4,22-03-30,216.00,111.00,10463.17,1.11600,1929.4,4602.45,83.4701
5,22-03-31,242.48,105.65,10442.23,1.10725,1943.1,4530.41,81.7081
6,22-04-01,251.40,104.75,10373.89,1.10470,1928.5,4545.86,85.0000


In [3]:
rates <- as.xts(as.data.frame(lapply(as.data.frame(lapply(Data[2:dim(Data)[2]], log)), diff)),
                    order.by = Data$DATE[2:dim(Data)[1]])
Data <- as.data.frame(rates)
head(Data)

,GAZP,BZ,CA,EURUSD,GC,SPX,USDRUB
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
22-03-25,-0.129984360,0.014946957,-0.007329153,-0.002391965,-0.001990254,0.005053402,-0.009393765
22-03-29,-0.087411938,-0.100022047,0.014604980,0.009876400,-0.017832767,0.019301587,-0.174941449
22-03-30,0.037740328,0.022227186,-0.002735404,0.006201988,0.003322607,-0.006313610,-0.018162773
22-03-31,0.115640825,-0.049398457,-0.002003692,-0.007871400,0.007075562,-0.015776327,-0.021335344
22-04-01,0.036126063,-0.008555185,-0.006566429,-0.002305659,-0.007542137,0.003404486,0.039498116
22-04-04,0.005948858,0.030369466,0.017171873,-0.006548051,0.001813237,0.008058323,-0.032887867


In [4]:
regrGazp <-  lm(Data$GAZP ~ Data$BZ + Data$CA + Data$EURUSD + Data$GC + Data$SPX + Data$USDRUB, data=Data)
summary(regrGazp)


Call:
lm(formula = Data$GAZP ~ Data$BZ + Data$CA + Data$EURUSD + Data$GC + 
    Data$SPX + Data$USDRUB, data = Data)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.35563 -0.00725 -0.00024  0.00786  0.22867 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)
(Intercept) -0.000922   0.001358  -0.679    0.498
Data$BZ      0.063828   0.065445   0.975    0.330
Data$CA      0.093515   0.120248   0.778    0.437
Data$EURUSD  0.047031   0.308784   0.152    0.879
Data$GC      0.198609   0.176242   1.127    0.260
Data$SPX    -0.051943   0.127020  -0.409    0.683
Data$USDRUB -0.029759   0.065713  -0.453    0.651

Residual standard error: 0.02944 on 465 degrees of freedom
Multiple R-squared:  0.01251,	Adjusted R-squared:  -0.0002365 
F-statistic: 0.9814 on 6 and 465 DF,  p-value: 0.4371


p-value = 0.4371 > 0.05 у F-статистики, что значит, что мы отвергаем 0-гипотезу о том, что данные справа значимы для доходностей Газпрома. Т.е. модель является вырожденной.